In [1]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 29.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 34.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 KB 8.5 MB/s eta 0:00:00a 0:00:01
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

/home/fulim/projects/Storyteller/api/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",  # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False,  # prevents running custom model files on your machine
                                             revision="main")  # which version of model to use in repo

/home/fulim/projects/Storyteller/api/myenv/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/home/fulim/projects/Storyteller/api/myenv/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/fulim/projects/Storyteller/api/myenv/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.
/home/fulim

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [5]:
model.eval()  # model in evaluation mode (dropout modules are deactivated)

# set padding token to end of string token
tokenizer.pad_token = tokenizer.eos_token

story = "Create a story with genre, title, characters and story"
prompt = f'''[INST] {story} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] Create a story with genre, title, characters and story [/INST] Title: The Enchanted Quill: A Tale of Magic and Ink

Genre: Fantasy

Story:

In the mystical land of Inkasia, nestled between the mountains and the sea, there was a legendary artifact known as the Enchanted Quill. This quill was said to possess the power to bring any words written with it to life. The people of Inkasia lived in awe of this magical tool, and many a tale had been spun about its origins and the adventures of those who wielded it.

Our story begins with the humble scribe, Elian, who lived in


In [6]:
prompt = """[INST] Create a story based on the given genre and title.
input:
    "Genre: Romance"
    "Title: Love Story Between James and Lucy" [/INST]
"output:
    "Characters: <Generate the characters based on title (if provided, if not based on your creativity)
    "Story" <Generate stories based on genre, title and characters>
Stop Here, Only 1 story is enough, generate as long as you need, and only return the story
"""

# set padding token to end of string token
tokenizer.pad_token = tokenizer.eos_token

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [ ]:
model.train()  # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from datasets import load_dataset

# Load dataset with specified split
dataset = load_dataset('FareedKhan/1k_stories_100_genre', split="train")

# Shuffle and select a range
dataset = dataset.shuffle(seed=42).select(range(10))

# Load NER pipeline
model_name = "dslim/bert-base-NER"  # Pre-trained NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline(
    "ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

# Function to extract characters using NER


def extract_characters(example):
    story = example['story']
    entities = ner_pipeline(story)  # Run NER on the story
    # Filter out only PERSON entities and ensure uniqueness
    characters = list(
        set(entity['word'] for entity in entities if entity['entity_group'] == 'PER'))
    return {"characters": characters}


# Apply the function to add the characters column
dataset = dataset.map(extract_characters)

# Function to format the prompt


def format_prompt_alpaca(example):
    """
    Create a structured and guiding prompt for story generation in Alpaca format,
    ensuring characters and story are both part of the output.
    """
    # Extract data from the example
    genre = example["genre"]
    title = example["title"]
    characters = ", ".join(example["characters"])
    story = example["story"]

    # Create the Alpaca-style prompt with instruction and input-output fields
    prompt = (
        "instruction: Generate a story based on the provided genre, title. Ensure the output includes the characters and the story."
        "input: ("
        f"Genre: {genre}\n"
        f"Title: {title}\n"
        "),"
        "output: ("
        f"Characters: {characters}\n"
        f"Story: {story}"
        ")"
    )

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        prompt,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs


# Apply formatting and retain only {"text": prompt}
dataset = dataset.map(format_prompt_alpaca, batched=True)

In [ ]:
# Inspect the final dataset
print(dataset)

In [ ]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer, mlm=False)

In [ ]:

# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir="stories-result",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

In [ ]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)

# train model
# silence the warnings. Please re-enable for inference!
model.config.use_cache = False
trainer.train()

# renable warnings
model.config.use_cache = True

In [ ]:
prompt = """[INST] Create a story based on the given genre and title.
input:
    "Genre: Romance"
    "Title: Love Story Between James and Lucy" [/INST]
"output:
    "Characters: <Generate the characters based on title (if provided, if not based on your creativity)
    "Story" <Generate stories based on genre, title and characters>
Stop Here, Only 1 story is enough, generate as long as you need, and only return the story
"""

In [ ]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])